# Preliminaries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import sys
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

sys.path.append("/Users/paolo/Documents/methods/CMI_FS")
from feature_selection import forwardFeatureSelection

sys.path.append("/Users/paolo/Documents/methods/LinCFA")
from LinCFA import LinCFA

sys.path.append("/Users/paolo/Documents/methods/NonLinCFA")
from NonLinCFA import NonLinCFA

sys.path.append("/Users/paolo/Documents/Droughts/Paolo/regression_NonLinCFA")
from aux import standardize,unfold_dataset,compute_r2,prepare_target,prepare_features,aggregate_unfolded_data,aggregate_unfolded_data_onlyTrain,FS_with_linearWrapper,compare_methods, compute_r2, aggregate_data_withoutUnfolding


# Piemonte Nord e Sud, Adda, Ticino, Dora

In [3]:
target_df_trainVal

,date,mean,median,year,week,mean_std
442,2010-09-11,0.684709,0.70,2010,36,1.532962
443,2010-09-19,0.660848,0.68,2010,37,1.368073
444,2010-09-27,0.636731,0.66,2010,39,1.201416
445,2010-10-05,0.625311,0.64,2010,40,1.122498
446,2010-10-13,0.600046,0.61,2010,41,0.947904
...,...,...,...,...,...,...
771,2017-11-25,0.240112,0.23,2017,47,-1.539395
772,2017-12-03,0.290279,0.28,2017,48,-1.192723
773,2017-12-11,0.421128,0.43,2017,50,-0.288497
774,2017-12-19,0.496628,0.50,2017,51,0.233241


In [3]:
### BOTH
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Piemonte_Nord','Piemonte_Sud','Adda','Dora','Ticino']
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    target_df_train = target_df_train.loc[target_df_train.date>='2010-09-10'].reset_index()
    target_df_trainVal = target_df_trainVal.loc[target_df_trainVal.date>='2010-09-10'].reset_index()
    
    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
        
    output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01')
        
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 91)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        

####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.270779
1    2001-01-13  0.494910    0.51  2001     2  0.221373
2    2001-01-21  0.496092    0.51  2001     3  0.229535
3    2001-01-29  0.427992    0.43  2001     5 -0.241061
4    2001-02-06  0.400512    0.41  2001     6 -0.430961
..          ...       ...     ...   ...   ...       ...
680  2015-11-28  0.526974    0.54  2015    48  0.442946
681  2015-12-06  0.522033    0.53  2015    49  0.408799
682  2015-12-14  0.400824    0.40  2015    51 -0.428806
683  2015-12-22  0.306771    0.31  2015    52 -1.078753
684  2015-12-30  0.305631    0.32  2015    53 -1.086628

[685 rows x 6 columns]
 target shapes: ((685, 6), (91, 6), (776, 6), (91, 6))
Number of features: 89

Number of aggregated features: 2

Number of features: 89

Number of aggregated features: 4

Number of features: 89

Number of aggregated features: 5

Numbe

----- MI Scores -----
[(12, 0.18402059176849156), (13, 0.1815332804821455), (14, 0.17861516528042667), (15, 0.17359646647132587), (9, 0.1664498310459468), (10, 0.1650301545095508), (16, 0.16355671042049524), (11, 0.15941316325431196), (8, 0.14968458885698102), (47, 0.1368603933623561), (48, 0.12539893875090236), (46, 0.12465458483517121), (44, 0.12462872600098943), (7, 0.12024460876389428), (17, 0.1171807635079893), (23, 0.1171803222858678), (49, 0.11490421559169771), (0, 0.1145710570391087), (1, 0.1111619582615638), (45, 0.1110259441778974), (5, 0.10784292252313497), (18, 0.10162044617441893), (4, 0.10117191304188047), (51, 0.09987732969445424), (3, 0.09744515149239599), (37, 0.09252769086322056), (6, 0.08996233765769437), (2, 0.088220610739348), (27, 0.0879181082227403), (25, 0.08697455450754296), (24, 0.08691589208468277), (40, 0.08563868685841462), (19, 0.08287919389094354), (42, 0.08148761034216159), (39, 0.08145161507354833), (50, 0.08101301336325983), (52, 0.07853855209812806), 

CMI: 0.055717492869203245
CMI: 0.07584292810852936
CMI: 0.007333183112108288
CMI: 0.031758556218146966
Highest CMI score: 0.07584292810852936
Adding original feature: 51
CMI: 0.0029105021321851887
CMI: 0.0031488460031872667
CMI: 0.016187744109625035
CMI: 0.009846384749772508
CMI: 0.0005450794172078832
Highest CMI score: 0.016187744109625035
Adding original feature: 34
CMI: 0.004697906698176824
CMI: 0.005091243371648457
Highest CMI score: 0.005091243371648457
Adding original feature: 50
CMI: 0.004036654644817839
CMI: 0.011538175148634089
Highest CMI score: 0.011538175148634089
Adding original feature: 15
Highest CMI score: -0.0038041352684772267

[13, 51, 34, 50, 15]


Full model and selected features with CMI

Full aggregate regression train score: 0.6103370880408033, test score: -0.3758582061391831
Aggregate regression train score with FS: 0.42666996674874114, test score: 0.3639018105315436

Full model and best 5 selected features with CMI

Full aggregate regression train score: 0.610

----- MI Scores -----
[(5, 0.20932196007363169), (6, 0.197724408490081), (0, 0.16570413508643198), (1, 0.13212400781578976), (3, 0.1280315595826143), (10, 0.12187249698671178), (4, 0.10992535752735562), (9, 0.1082331069306501), (30, 0.1041592896184971), (7, 0.09840467592005793), (2, 0.09730653341573577), (31, 0.08010581464226418), (8, 0.07651690952768497), (32, 0.06615794112589012), (14, 0.06442595899910247), (15, 0.058237819534182333), (12, 0.057981658672315174), (39, 0.054459538563835806), (37, 0.0543107649689457), (13, 0.04863370718818426), (11, 0.0485963968184967), (27, 0.042529009851189445), (21, 0.041025059369700106), (20, 0.03742359539264455), (35, 0.03703425845505514), (34, 0.03690914849625687), (26, 0.0355387482402489), (25, 0.0352495441733584), (19, 0.03152047254753106), (18, 0.02940560041405286), (23, 0.028957121769385717), (24, 0.024933559090087827), (33, 0.018796897823642512), (36, 0.01843123792030893), (17, 0.016042612608089757), (28, 0.014680021442254502), (16, 0.0112506

## Only snow

In [17]:
target_df_train

,date,mean,median,year,week,mean_std
0,2001-01-05,0.278983,0.00,2001,1,-1.146332
1,2001-01-13,0.494910,0.51,2001,2,0.371173
2,2001-01-21,0.496092,0.51,2001,3,0.379474
3,2001-01-29,0.427992,0.43,2001,5,-0.099118
4,2001-02-06,0.400512,0.41,2001,6,-0.292244
...,...,...,...,...,...,...
406,2009-11-27,0.363952,0.37,2009,48,-0.549184
407,2009-12-05,0.400487,0.40,2009,49,-0.292423
408,2009-12-13,0.506771,0.52,2009,50,0.454529
409,2009-12-21,0.387530,0.53,2009,52,-0.383480


In [10]:
### BOTH
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Adda','Dora','Piemonte_Nord','Piemonte_Sud','Ticino']
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
path_features_snow='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/snow_aggreg_IT-SNOW/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    target_df_train = target_df_train.loc[target_df_train.date>='2010-09-10'].reset_index()
    target_df_trainVal = target_df_trainVal.loc[target_df_trainVal.date>='2010-09-10'].reset_index()
    
    eps = 0.001
    snow_actual_path = path_features_snow+basin+'_aggreg_snow_allCoord.csv'
    
    if ((basin=='Piemonte_Nord') | (basin=='Piemonte_Sud')):    
        output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_snow', 
                                                                             'cyclostationary_mean_snow_1w',
                                                                             'cyclostationary_mean_snow_4w', 
                                                                             'cyclostationary_mean_snow_8w',
                                                                             'cyclostationary_mean_snow_12w', 
                                                                             'cyclostationary_mean_snow_16w',
                                                                             'cyclostationary_mean_snow_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01')
    else: 
        output,aggregate_trainVal,aggregate_test = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS', 
                                                                             'cyclostationary_mean_HS_1w',
                                                                             'cyclostationary_mean_HS_4w', 
                                                                             'cyclostationary_mean_HS_8w',
                                                                             'cyclostationary_mean_HS_12w', 
                                                                             'cyclostationary_mean_HS_16w',
                                                                             'cyclostationary_mean_HS_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01') 
        
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 91)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
        

####################Adda####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.039373    0.00  2001     1 -2.636613
1    2001-01-13  0.380618    0.43  2001     2 -0.371379
2    2001-01-21  0.341985    0.38  2001     3 -0.627832
3    2001-01-29  0.322044    0.35  2001     5 -0.760200
4    2001-02-06  0.354954    0.40  2001     6 -0.541742
..          ...       ...     ...   ...   ...       ...
680  2015-11-28  0.570366    0.62  2015    48  0.888198
681  2015-12-06  0.534491    0.57  2015    49  0.650052
682  2015-12-14  0.388480    0.41  2015    51 -0.319187
683  2015-12-22  0.348693    0.38  2015    52 -0.583303
684  2015-12-30  0.336782    0.37  2015    53 -0.662372

[685 rows x 6 columns]
 target shapes: ((685, 6), (91, 6), (776, 6), (91, 6))
Number of features: 118

Number of aggregated features: 41

Number of features: 118

Number of aggregated features: 42

Number of features: 118

Number of aggregated features: 43

Number o

CMI: 0.009854222917041383
CMI: 0.01415465443516098
CMI: 0.017899572833935007
CMI: 0.007193775635605221
CMI: 0.01344288056815382
CMI: 0.029779481994635093
CMI: 0.024486690793048382
CMI: 0.0006825152051516126
CMI: 0.011082393022484538
CMI: 0.007526261870662743
CMI: 0.0006575957417593592
CMI: 0.009670940430013097
CMI: 0.01745361795036271
CMI: 0.0025774096266510393
CMI: 0.0029656461573197845
CMI: 0.021790200248282487
CMI: 0.008326780238742698
Highest CMI score: 0.029779481994635093
Adding original feature: 110
CMI: 0.011341412912957455
CMI: 0.010666907858960517
CMI: 0.004028722271286755
CMI: 0.009100525310051316
CMI: 0.014290573474775053
CMI: 0.0026451090470547556
CMI: 0.005467452897371827
CMI: 0.004666792296755512
CMI: 0.02592418870602481
CMI: 0.025334001727015637
CMI: 0.0019917913191963843
CMI: 0.0006927146310167498
CMI: 0.013381727453583014
CMI: 0.0009908416968378297
CMI: 0.031199514453049304
CMI: 0.029901702469518815
CMI: 0.0038453413411919934
CMI: 1.7953507110674005e-05
CMI: 0.0116114

Number of features: 56

Number of aggregated features: 23

Number of features: 56

Number of aggregated features: 23

Number of features: 56

Number of aggregated features: 23

Number of features: 56

Number of aggregated features: 23

Number of features: 56

Number of aggregated features: 26

Number of features: 56

Number of aggregated features: 27

Number of features: 56

Number of aggregated features: 24



selected columns: ['cyclostationary_mean_HS_4w_13', 'cyclostationary_mean_HS_16w_8', 'cyclostationary_mean_HS_24w_23', 'cyclostationary_mean_HS_12w_25', 'cyclostationary_mean_HS_16w_26', 'cyclostationary_mean_HS_8', 'cyclostationary_mean_HS_16w_15', 'cyclostationary_mean_HS_7', 'cyclostationary_mean_HS_12w_8', 'cyclostationary_mean_HS_16w_17', 'cyclostationary_mean_HS_24w_1', 'cyclostationary_mean_HS_24w_18', 'cyclostationary_mean_HS_12w_0', 'cyclostationary_mean_HS_24w_11', 'cyclostationary_mean_HS_0', 'cyclostationary_mean_HS_1w_22', 'cyclostationary_mean_HS_1w_2', 'cyclostati

Number of features: 97

Number of aggregated features: 43

Number of features: 97

Number of aggregated features: 44

Number of features: 97

Number of aggregated features: 44

Number of features: 97

Number of aggregated features: 49

Number of features: 97

Number of aggregated features: 55

Number of features: 97

Number of aggregated features: 45

Number of features: 97

Number of aggregated features: 43



selected columns: ['cyclostationary_mean_snow_14', 'cyclostationary_mean_snow_12w_6', 'cyclostationary_mean_snow_24w_31', 'cyclostationary_mean_snow_16w_12', 'cyclostationary_mean_snow_24w_11', 'cyclostationary_mean_snow_4w_36', 'cyclostationary_mean_snow_13', 'cyclostationary_mean_snow_8w_31', 'cyclostationary_mean_snow_16w_44', 'cyclostationary_mean_snow_16w_8', 'cyclostationary_mean_snow_4w_12', 'cyclostationary_mean_snow_10', 'cyclostationary_mean_snow_12w_16', 'cyclostationary_mean_snow_4w_23', 'cyclostationary_mean_snow_1w_7', 'cyclostationary_mean_snow_41', 'cyclostationa

CMI: 0.010502633895139668
CMI: 0.01978608724991493
CMI: 0.006675900862409739
CMI: 0.004285677119013898
CMI: 0.0009460679344648337
CMI: 0.040006940686859116
CMI: 0.01774629555324328
CMI: 0.006750990159418907
CMI: 0.012504734251425759
CMI: 0.018428763006014426
CMI: 0.004218815070456933
CMI: 0.0592013313043733
CMI: 0.05832273849001787
CMI: 0.017935699105316727
CMI: 0.010983240594128085
CMI: 0.05454443026101766
CMI: 0.008354688803978244
CMI: 0.03391789585213695
CMI: 0.02734853022572084
CMI: 0.009292578445344618
Highest CMI score: 0.0592013313043733
Adding original feature: 83
CMI: 0.010502633895139807
CMI: 0.028347248561772898
CMI: 0.019786087249914985
CMI: 0.017246745648370893
CMI: 0.004285677119013925
CMI: 0.0009460679344649725
CMI: 0.01544043533697445
CMI: 0.018878881345112453
CMI: 0.02868696480874966
CMI: 0.013637320043294987
CMI: 0.04023347209906189
CMI: 0.005366164273382545
CMI: 0.0208460726059915
CMI: 0.027785320095607885
CMI: 0.03499674039806916
CMI: 0.006107513406820947
CMI: 0.016

CMI: 0.0066891395192526515
CMI: 0.01185307695732199
CMI: 0.00808803320336765
CMI: 0.013862081889593425
CMI: 0.009004378143320363
CMI: 0.00693967912968857
CMI: 0.005301403346836708
CMI: 0.011226506254297969
CMI: 0.018734450958800497
CMI: 0.03052629524594605
CMI: 0.006349570307967456
CMI: 0.002122142548852207
CMI: 0.008715979960204595
CMI: 0.009112270075606638
CMI: 0.0008316342915528352
CMI: 0.024772091844279587
CMI: 0.011237468663435307
CMI: 0.008332434711710135
CMI: 0.011752518572851423
CMI: 0.0005152139189239513
CMI: 0.024849166625245694
CMI: 0.020851097803197688
CMI: 0.002119993332120401
CMI: 0.0021380315052908494
CMI: 0.03278135147620928
CMI: 0.017469918326232042
CMI: 0.01810479370259699
CMI: 0.007139071864912194
CMI: 0.005151273978536763
CMI: 0.020051176230742013
CMI: 0.008965808085547117
CMI: 0.030757210032413884
CMI: 0.006127888037612855
CMI: 0.009378397652987924
CMI: 0.015007037664818396
CMI: 0.02740634114494009
CMI: 0.0018409719877431896
CMI: 0.016883938956496397
CMI: 0.0022073

CMI: 0.003854888406948309
CMI: 0.002564817530987029
CMI: 0.0017639689185602325
CMI: 3.1405372790949126e-05
CMI: 0.0011372543146849767
CMI: 0.006286041828045208
CMI: 0.007559691288542336
CMI: 0.009944011645718676
CMI: 0.0011401628002539743
CMI: 0.007006103886151027
CMI: 0.010276254072299373
CMI: 0.007704175499018362
CMI: 0.0023710303002008604
CMI: 0.008402365404090617
CMI: 0.007793306474007888
CMI: 0.0010612307187750636
CMI: 0.0072984769565248575
CMI: 0.00218136174941147
CMI: 0.0002112959244939261
CMI: 0.024683235081015886
CMI: 0.0015985012982461244
CMI: 0.02712959050474517
CMI: 0.008253697963161333
CMI: 0.009417730240174838
CMI: 0.009482813621783659
CMI: 0.002147856181643093
CMI: 0.005921907666463888
CMI: 0.0022908205540273507
CMI: 0.003094345903003781
CMI: 0.001526699481189664
CMI: 0.0009345376329257249
CMI: 0.0058752132835547255
CMI: 0.005095892649532807
Highest CMI score: 0.02712959050474517
Adding original feature: 405
CMI: 0.009964684485554542
CMI: 0.01041556665080351
CMI: 0.00045

Highest CMI score: 0.0

[392, 295, 370, 391, 319, 315, 18, 284, 205, 405, 139, 89, 311, 221, 55, 199, 198, 212, 183, 326, 314, 279, 404, 256]


Full model and selected features with CMI

Full aggregate regression train score: 1.0, test score: -17520.34673404912
Aggregate regression train score with FS: 0.08748298029850698, test score: -2.7702397427860843

Full model and best 5 selected features with CMI

Full aggregate regression train score: 1.0, test score: -17520.34673404912
Aggregate regression train score with FS: 0.009213697196216675, test score: -0.09980407071611297
####################Ticino####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.264043    0.00  2001     1 -1.161265
1    2001-01-13  0.354618    0.39  2001     2 -0.504299
2    2001-01-21  0.427990    0.47  2001     3  0.027896
3    2001-01-29  0.339495    0.35  2001     5 -0.613987
4    2001-02-06  0.324134    0.34  2001     6 -0.725408
..          ...      

CMI: 0.0027801075452481283
CMI: 0.03385807929508103
CMI: 0.04823981286310455
CMI: 0.035704653964216265
CMI: 0.008334660663609522
CMI: 0.03404229183331353
CMI: 0.0064101205429044905
CMI: 0.03382253993079734
CMI: 0.02328354042843711
CMI: 0.018064179660967683
CMI: 0.01099272070718263
CMI: 0.014093597245962097
CMI: 0.012647382891629477
CMI: 0.013709704714841231
CMI: 0.01776368546153645
CMI: 0.015028560817870018
CMI: 0.0204440285936211
CMI: 0.017904496062475184
CMI: 0.03980642676385823
CMI: 0.030057984918475167
CMI: 0.02413857519988266
CMI: 0.019407797905619445
CMI: 0.017926774264883816
CMI: 0.017039000924247927
CMI: 0.027982800571644345
CMI: 0.014908935869032303
CMI: 0.0038158837919713418
CMI: 0.055034667549642174
CMI: 0.024840107568366937
CMI: 0.03780289129434514
CMI: 0.0031171838358937387
CMI: 0.024796782815491997
CMI: 0.04668369121414731
Highest CMI score: 0.06803084095103193
Adding original feature: 1
CMI: 0.0041864480520760194
CMI: 0.011064905255580793
CMI: 0.0006073332121106501
CMI: 

## Temp + Prec + Snow

In [11]:
### Without 12 months
#basins = ['Adda','Dora','Emiliani1','Emiliani2','Garda_Mincio','Lambro_Olona','Oglio_Iseo','Piemonte_Nord','Piemonte_Sud','Ticino']
basins = ['Piemonte_Nord','Piemonte_Sud','Adda','Dora','Ticino']
path_target = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/csv_VHI/'
path_features_snow='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/snow_aggreg_IT-SNOW/'
path_features='/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/features/csv_allvalues/temporal_aggreg/'

for basin in basins:
    print('####################' + basin + '####################')

    target_df_train,target_df_val,target_df_test,target_df_trainVal = prepare_target('',max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01', path=path_target+basin+'.csv')
    target_df_train = target_df_train.loc[target_df_train.date>='2010-09-10'].reset_index()
    target_df_trainVal = target_df_trainVal.loc[target_df_trainVal.date>='2010-09-10'].reset_index()
    
    eps = 0.001
    actual_path = path_features+basin+'_aggreg.csv'
    snow_actual_path = path_features_snow+basin+'_aggreg_snow_allCoord.csv'    
    
    output,aggregate_trainVal_temp_prec,aggregate_test_temp_prec = aggregate_unfolded_data(actual_path,['cyclostationary_mean_tg', 
                                                                             'cyclostationary_mean_tg_1w',
                                                                             'cyclostationary_mean_tg_4w', 
                                                                             'cyclostationary_mean_tg_8w',
                                                                             'cyclostationary_mean_tg_12w', 
                                                                             'cyclostationary_mean_tg_16w',
                                                                             'cyclostationary_mean_tg_24w',
                                                                             'cyclostationary_mean_rr', 
                                                                             'cyclostationary_mean_rr_1w',
                                                                             'cyclostationary_mean_rr_4w', 
                                                                             'cyclostationary_mean_rr_8w',
                                                                             'cyclostationary_mean_rr_12w', 
                                                                             'cyclostationary_mean_rr_16w',
                                                                             'cyclostationary_mean_rr_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01')
    
    if ((basin=='Piemonte_Nord') | (basin=='Piemonte_Sud')):    
        output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_snow', 
                                                                             'cyclostationary_mean_snow_1w',
                                                                             'cyclostationary_mean_snow_4w', 
                                                                             'cyclostationary_mean_snow_8w',
                                                                             'cyclostationary_mean_snow_12w', 
                                                                             'cyclostationary_mean_snow_16w',
                                                                             'cyclostationary_mean_snow_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01')
    else: 
        output_snow,aggregate_trainVal_snow,aggregate_test_snow = aggregate_unfolded_data(snow_actual_path,['cyclostationary_mean_HS', 
                                                                             'cyclostationary_mean_HS_1w',
                                                                             'cyclostationary_mean_HS_4w', 
                                                                             'cyclostationary_mean_HS_8w',
                                                                             'cyclostationary_mean_HS_12w', 
                                                                             'cyclostationary_mean_HS_16w',
                                                                             'cyclostationary_mean_HS_24w'
                                                                            ],
                                                                       target_df_trainVal, eps=eps,
                                                                       max_train='2016-01-01', max_val='2018-01-01', max_test='2020-01-01') 
        
    
    aggregate_trainVal = pd.concat((aggregate_trainVal_snow,aggregate_trainVal_temp_prec),axis=1)
    aggregate_test = pd.concat((aggregate_test_snow,aggregate_test_temp_prec),axis=1)
    
    selected_colnames = FS_with_linearWrapper(aggregate_trainVal, target_df_train, target_df_val, min(50,aggregate_trainVal.shape[1]-1), 91)

    print('\nFull model and selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with wrapper\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    train_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_wrapper_best5_train_withSnow.csv'
    val_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_wrapper_best5_val_withSnow.csv'
    test_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_wrapper_best5_test_withSnow.csv'
        
    #aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    #aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    #aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)
    
    res = {
            "delta" : [], 
            "numSelected" : [], 
            "selectedFeatures" : [] 
        }
        
    res['selectedFeatures'] = forwardFeatureSelection(10,np.array(aggregate_trainVal),np.array(target_df_trainVal.mean_std),res,10,1)
        
    selectedFeatures='selectedFeatures'
    print(f'\n{res[selectedFeatures]}\n')
    selected_colnames = aggregate_trainVal.columns[res['selectedFeatures']]
    
    print('\nFull model and selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames)
    
    print('\nFull model and best 5 selected features with CMI\n')
    compare_methods(aggregate_trainVal, aggregate_test, target_df_trainVal, target_df_test, selected_colnames[0:5])
    
    train_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_best5_CMI_train_withSnow.csv'
    val_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_best5_CMI_val_withSnow.csv'
    test_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_best5_CMI_test_withSnow.csv'
            
    #aggregate_trainVal.loc[:410,selected_colnames[0:5]].to_csv(train_string, index=False)
    #aggregate_trainVal.loc[411:,selected_colnames[0:5]].to_csv(val_string, index=False)
    #aggregate_test.loc[:,selected_colnames[0:5]].to_csv(test_string, index=False)

    train_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_CMI_train_withSnow.csv'
    val_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_CMI_val_withSnow.csv'
    test_string = '/Users/paolo/Documents/OneDrive - Politecnico di Milano/droughts/reduced_features/'+basin+'_nonLinCFA_CMI_test_withSnow.csv'
        
    #aggregate_trainVal.loc[:410,selected_colnames].to_csv(train_string, index=False)
    #aggregate_trainVal.loc[411:,selected_colnames].to_csv(val_string, index=False)
    #aggregate_test.loc[:,selected_colnames].to_csv(test_string, index=False)
        

####################Piemonte_Nord####################
target samples:            date      mean  median  year  week  mean_std
0    2001-01-05  0.278983    0.00  2001     1 -1.270779
1    2001-01-13  0.494910    0.51  2001     2  0.221373
2    2001-01-21  0.496092    0.51  2001     3  0.229535
3    2001-01-29  0.427992    0.43  2001     5 -0.241061
4    2001-02-06  0.400512    0.41  2001     6 -0.430961
..          ...       ...     ...   ...   ...       ...
680  2015-11-28  0.526974    0.54  2015    48  0.442946
681  2015-12-06  0.522033    0.53  2015    49  0.408799
682  2015-12-14  0.400824    0.40  2015    51 -0.428806
683  2015-12-22  0.306771    0.31  2015    52 -1.078753
684  2015-12-30  0.305631    0.32  2015    53 -1.086628

[685 rows x 6 columns]
 target shapes: ((685, 6), (91, 6), (776, 6), (91, 6))
Number of features: 89

Number of aggregated features: 2

Number of features: 89

Number of aggregated features: 4

Number of features: 89

Number of aggregated features: 5

Numbe

CMI: 0.055428199933395456
CMI: 0.014865744498314709
CMI: 0.035544603802494856
CMI: 0.013615945445693556
CMI: 0.04319862806927302
CMI: 0.045763280906802206
CMI: 0.09592466718755771
CMI: 0.03609429274624412
CMI: 0.023164883930189034
CMI: 0.07662907130323346
CMI: 0.041233502761736374
CMI: 0.027354540297700264
CMI: 0.023198574486645418
CMI: 0.02498047937214906
CMI: 0.04688000088568281
CMI: 0.03447322764337851
CMI: 0.02839365756881318
CMI: 0.03609456619727097
CMI: 0.0007642722724515161
CMI: 0.010270332778523433
CMI: 0.042206084530556215
CMI: 0.026986313720873123
CMI: 0.057225447752476594
CMI: 0.059961795807008555
CMI: 0.11789936997180661
CMI: 0.07797530886580403
CMI: 0.07333102365666155
CMI: 0.04180262151718103
CMI: 0.018435373459215643
CMI: 0.010909769644455614
CMI: 0.003292503656343315
CMI: 0.00873831160558139
CMI: 0.03060366701640735
CMI: 0.013168635523995137
CMI: 0.030496558603913926
CMI: 0.02345651042232677
CMI: 0.05041983060461999
CMI: 0.029731231256525936
CMI: 0.04543178785569921
CMI

CMI: 0.006738915813341462
CMI: 0.00466082159158554
CMI: 0.007601499918793886
CMI: 0.007386574685476155
CMI: 0.004587022508037886
CMI: 0.00279723438256263
CMI: 0.023875593162745457
CMI: 0.021880778216954444
CMI: 0.010290315790864979
CMI: 0.015188877013947388
CMI: 0.0025707531353455493
CMI: 0.006954904327639844
CMI: 0.007513816371254645
CMI: 0.006038154329674805
Highest CMI score: 0.023875593162745457
Adding original feature: 81
CMI: 0.0013120382103978256
CMI: 0.006136577093620643
CMI: 0.015900293157610823
CMI: 7.513676190479268e-05
CMI: 0.0025377436305819012
Highest CMI score: 0.015900293157610823
Adding original feature: 83
CMI: 0.017343575489271124
CMI: 0.015236913869696334
CMI: 0.004151845321858805
CMI: 0.008700161022804298
CMI: 0.024497747401630243
CMI: 0.0010660310601817713
CMI: 0.007137734219166414
CMI: 0.008944647775582237
CMI: 0.007328297834584829
CMI: 0.014128931019945057
CMI: 0.002233086099526993
CMI: 0.013116721295641787
CMI: 0.019448635572945516
CMI: 0.011379862839585642
CMI

CMI: 0.0066891395192526515
CMI: 0.01185307695732199
CMI: 0.00808803320336765
CMI: 0.013862081889593425
CMI: 0.009004378143320363
CMI: 0.00693967912968857
CMI: 0.005301403346836708
CMI: 0.011226506254297969
CMI: 0.018734450958800497
CMI: 0.03052629524594605
CMI: 0.006349570307967456
CMI: 0.002122142548852207
CMI: 0.008715979960204595
CMI: 0.009112270075606638
CMI: 0.0008316342915528352
CMI: 0.024772091844279587
CMI: 0.011237468663435307
CMI: 0.008332434711710135
CMI: 0.011752518572851423
CMI: 0.0005152139189239513
CMI: 0.024849166625245694
CMI: 0.020851097803197688
CMI: 0.002119993332120401
CMI: 0.0021380315052908494
CMI: 0.03278135147620928
CMI: 0.017469918326232042
CMI: 0.01810479370259699
CMI: 0.007139071864912194
CMI: 0.005151273978536763
CMI: 0.020051176230742013
CMI: 0.008965808085547117
CMI: 0.030757210032413884
CMI: 0.006127888037612855
CMI: 0.009378397652987924
CMI: 0.015007037664818396
CMI: 0.02740634114494009
CMI: 0.0018409719877431896
CMI: 0.016883938956496397
CMI: 0.0022073

CMI: 0.0029565996937009897
CMI: 0.004562359419666817
CMI: 0.015846635289291533
CMI: 0.0077858835990242126
CMI: 0.0001903036248780965
CMI: 0.0014309021088030893
CMI: 0.014584698756285486
CMI: 0.008612407417124879
CMI: 0.0038612459438429614
CMI: 0.004735586006963277
CMI: 0.005179917036606918
CMI: 0.00723143243139468
CMI: 0.016385183319730712
CMI: 0.004798182235453696
CMI: 0.007830835600343478
CMI: 0.0028506244610748688
CMI: 0.00129497172850912
CMI: 0.006954674628368018
CMI: 0.008314293075511181
CMI: 0.013867420096735084
CMI: 0.0069333120458041075
CMI: 0.007276413722253572
CMI: 0.010916552736640717
CMI: 0.010820854686220016
CMI: 0.007710134852859818
CMI: 0.00824162173794929
CMI: 0.0033294540928150984
CMI: 0.006237078958726772
CMI: 0.003491198748226454
CMI: 0.003433170138083863
CMI: 0.005131476209834451
CMI: 0.001750959697456922
CMI: 0.010080687075342265
CMI: 0.021194109948419082
CMI: 0.011182793335250352
CMI: 0.002427909993837518
CMI: 0.0006835354225148693
Highest CMI score: 0.02387546430

...

----- MI Scores -----
[(332, 0.1926990877376534), (323, 0.1907894516441214), (333, 0.18733794988323108), (330, 0.1856089519960697), (11, 0.17348053767637692), (334, 0.17261674346379952), (322, 0.16957930324112577), (335, 0.1642648270268545), (331, 0.15996592393559178), (110, 0.159707222341812), (127, 0.1576588923621339), (336, 0.15694317765658466), (320, 0.15119724112278232), (206, 0.1510057778551819), (126, 0.14795757853415087), (26, 0.14783361751941235), (112, 0.14519616391285245), (231, 0.14495127585051268), (232, 0.1448110001315959), (202, 0.14407127280439871), (137, 0.14136827343063574), (174, 0.1409816190045876), (147, 0.13999506001920367), (251, 0.13975908442490745), (85, 0.13776245042263466), (29, 0.13713010443057702), (207, 0.1361913629945041), (238, 0.13617658557415127), (53, 0.13548770215105224), (183, 0.1354000708714773), (173, 0.13473213043636073), (107, 0.13432592573877078), (184, 0.13406618136189033), (139, 0.13332432517005735), (152, 0.13246880313586412), (248, 0.132347

CMI: 0.07137985381238063
CMI: 0.0070990484694422595
CMI: 0.029667969554739576
CMI: 0.024663717769318055
CMI: 0.024854432438257495
CMI: 0.01389427416688882
CMI: 0.11098133610247943
CMI: 0.0675147445436605
CMI: 0.037243159232149464
CMI: 0.03458945021615634
CMI: 0.034131217955723575
CMI: 0.08612005876311582
CMI: 0.057839456977458265
CMI: 0.10378778237615036
CMI: 0.05790845695336996
CMI: 0.02469485046028158
CMI: 0.02443272440348848
CMI: 0.07479271375749744
CMI: 0.09689899843898628
CMI: 0.0822805306817774
CMI: 0.0015925513939247238
CMI: 0.03636983060966692
CMI: 0.04300231917297101
CMI: 0.02359086163139318
CMI: 0.03667849467369011
CMI: 0.013320921795333202
CMI: 0.08955508680392013
CMI: 0.10099417145189557
CMI: 0.06054472232127975
CMI: 0.08649284424769141
CMI: 0.1251874211137187
CMI: 0.004958494839443273
CMI: 0.009831345108028577
CMI: 0.010357732706428008
CMI: 0.020144756670331443
CMI: 0.05195213204917806
CMI: 0.07548114790115287
CMI: 0.0755701376638602
CMI: 0.09652600092769595
CMI: 0.0945796

CMI: 0.055717492869203245
CMI: 0.07584292810852936
CMI: 0.007333183112108288
CMI: 0.031758556218146966
Highest CMI score: 0.14668118390165516
Adding original feature: 67
CMI: 0.030606185576777967
CMI: 0.03691914763035359
CMI: 0.014117740477872842
CMI: 0.00437743558595588
CMI: 0.0068764025531379835
CMI: 0.020368485840211836
CMI: 0.03356463438968671
CMI: 0.024404233160086553
CMI: 0.009876961204179469
CMI: 0.02513389610801403
CMI: 0.040662075303407486
CMI: 0.04342523680125049
CMI: 0.01646038086132351
CMI: 0.03249835600559864
CMI: 0.01796929773230327
CMI: 0.014108782577193002
CMI: 0.04708659094031592
CMI: 0.020056215252264975
CMI: 0.00031378087080374817
CMI: 0.021720782679187867
CMI: 0.02076402540140232
CMI: 0.000186961320933543
CMI: 0.04962207036494942
CMI: 0.05053726797080749
CMI: 0.05400274436286129
CMI: 0.006157860255567638
CMI: 0.034914579583295524
CMI: 0.02413091831246328
CMI: 0.0011557825863268145
CMI: 0.004254104310981499
CMI: 0.026471505669397988
CMI: 0.02491641480896789
CMI: 0.00

CMI: 0.0180294364334736
CMI: 0.047032638559606266
CMI: 0.021954234473467837
CMI: 0.011445392144451227
CMI: 0.010388577908020169
CMI: 0.011763143251652952
CMI: 0.0058121886067449835
CMI: 0.030240537754999852
CMI: 0.020187316405522626
CMI: 0.013025390889372268
CMI: 0.004823444452830228
CMI: 0.014238541834920904
CMI: 0.03166213347634789
CMI: 0.029956368171829673
CMI: 0.01112281246009178
CMI: 0.0159009422355067
CMI: 0.0247725236328577
CMI: 0.0008313497000956294
CMI: 0.007484378500653699
CMI: 0.0036972025846553347
CMI: 0.013490359310395328
CMI: 0.02976762341080852
CMI: 0.02455322386725789
CMI: 0.00929670845384356
Highest CMI score: 0.047032638559606266
Adding original feature: 37
CMI: 0.0009512672715808468
CMI: 0.00720564450334249
Highest CMI score: 0.00720564450334249
Adding original feature: 137
CMI: 0.0005392866854401768
CMI: 0.004140568773183362
Highest CMI score: 0.004140568773183362
Adding original feature: 38
CMI: 0.007428032626833836
CMI: 0.014267785462615423
CMI: 0.0151681529464915

CMI: 0.03222642986570268
CMI: 0.06625336938491688
CMI: 0.057806681469170174
CMI: 0.028972702885245727
CMI: 0.02352702522817582
CMI: 0.033425494860745725
CMI: 0.04041394848766039
CMI: 0.007248336860092103
CMI: 0.0002477478677733458
CMI: 0.011119517789619615
CMI: 0.05851513555607102
CMI: 0.07463118935151325
CMI: 0.030474511531416892
CMI: 0.029657366860834172
CMI: 0.0637408604313151
CMI: 0.05465691023654029
CMI: 0.06557327227053183
CMI: 0.04117087704523431
CMI: 0.017839204361112998
CMI: 0.059232537516112405
CMI: 0.04801673371836701
CMI: 0.07152127475102354
CMI: 0.10812944181905931
CMI: 0.046326965334546555
CMI: 0.08693380606830176
CMI: 0.05612359560179009
CMI: 0.07338460070064567
CMI: 0.07374116498176558
CMI: 0.042551902666114694
CMI: 0.023557240508018357
CMI: 0.03600032397595693
CMI: 0.03081942057360848
CMI: 0.04286895397928775
CMI: 0.106597194334215
CMI: 0.08551238589157006
CMI: 0.06616226261136682
CMI: 0.03537267197170166
CMI: 0.06657504926332958
CMI: 0.034216468812939826
CMI: 0.105726